# Text versus Bytes

## Character Issues

#### Example 4-1. Encoding and Decoding

In [1]:
s = 'café'
len(s)

4

In [2]:
b = s.encode('utf8')
b

b'caf\xc3\xa9'

In [3]:
len(b) # é가 UTF-8로 2byte

5

In [4]:
b.decode('utf8')

'café'

## Byte Essentials

이진 시퀀스에 사용되는 2가지 내장 자료형. 1바이트 정수를 연속적으로 저장.
* bytes: 불변형
* bytearray: 가변형

#### Example 4-2. A five-byte sequence as bytes and as bytearray

In [5]:
cafe = bytes('café', encoding='utf_8')
cafe

b'caf\xc3\xa9'

In [6]:
cafe[0]

99

In [7]:
cafe[:1]

b'c'

In [8]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [9]:
cafe_arr[-1:]

bytearray(b'\xa9')

**cafe[0]은 정수가 cafe[:1]은 bytes가 반환되는 이유**  
cafe[i]는 항목 하나를, cafe[i:i+1]는 해당되는 시퀀스를 동일한 자료형으로 반환.  
bytes는 0-255사이의 정수가 들어간 시퀀스임.  
s[0] == s[:1]이 성립하는 시퀀스형은 str이 유일.

#### Example 4-3. Initializing bytes from the raw data of an array

In [12]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2]) # short int(16bit)
octets = bytes(numbers)
octets # 10 bytes

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

### Structs and Memory Views

#### Example 4-4. Using *memoryview* and *struct* to inpsect a GIF image header

In [15]:
import struct
fmt = '<3s3sHH'
with open('sample.gif', 'rb') as fp:
  img = memoryview(fp.read())

header = img[:10]
bytes(header)

b'GIF89a\xf4\x01\xef\x00'

In [16]:
struct.unpack(fmt, header)

(b'GIF', b'89a', 500, 239)

In [20]:
# momoryview를 슬라이싱하면 바이트를 복사하지 않고 새로운 memoryview 객체를 반환.
# 참조를 삭제해 memoryview 객체에 연결된 메모리 해제
del header
del img

NameError: ignored

## Basic Encoders/Decoders

#### Example 4-5. The String 'El Niño' encoded with three codecs producing very different byte sequences

In [21]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
  print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## Understanding Encode/Decode Problems

### Coping with UnicodeEncodeError

#### Example 4-6.  Encoding to bytes: success and error handling

In [22]:
city = 'São Paulo'
city.encode('utf_8')

b'S\xc3\xa3o Paulo'

In [23]:
city.encode('utf_16')

b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'

In [25]:
city.encode('iso8859_1')

b'S\xe3o Paulo'

In [26]:
city.encode('cp437')

UnicodeEncodeError: ignored

In [27]:
city.encode('cp437', errors='ignore')

b'So Paulo'

In [28]:
city.encode('cp437', errors='replace')

b'S?o Paulo'

In [29]:
city.encode('cp437', errors='xmlcharrefreplace')

b'S&#227;o Paulo'

### Coping with UnicodeDecodeError

#### Example 4-7. illustrates how using the wrong codec may produce gremlins or a UnicodeDecodeError


In [31]:
octets = b'Montr\xe9al'
octets.decode('cp1252')

'Montréal'

In [32]:
octets.decode('iso8859_7')

'Montrιal'

In [33]:
octets.decode('koi8_r')

'MontrИal'

In [34]:
octets.decode('utf_8')

UnicodeDecodeError: ignored

In [35]:
octets.decode('utf_8', errors='replace')

'Montr�al'

### SyntaxError When Loading Modules with Unexpected Encoding

#### Example 4-8. ola.py: 'Hellow, World' in Porttuguese

In [38]:
# coding: cp1252
print('Olá, Mundo!')

Olá, Mundo!


### How to Discover the Encoding of a Byte Sequence  

바이트 시퀀스의 인코딩 방식은 별도로 정보를 가져와야 알 수 있음. HTTP나 XML 같은 통신 프로토콜이나 파일포맷은 인코딩 방식을 명시하는 헤더를 포함하고 있음.  
바이트스트림의 값의 범위 또는 퍁패턴을 통해 인코딩 방식 추정 가능.  
*Chardet* 패키지를 이용해 30가지 인코딩 방식을 알아낼 수 있음.

### BOM: A Useful Gremlin


Example 4-5에서 utf16 인코딩 앞의 b'\xff\xfe'는 바이트 표기 순서 표시(BOM). 인코딩한 인텔 CPU의 '리틀엔디언' 바이트 순서를 나타냄.  

In [39]:
u16 = 'El Niño'.encode('utf_16')
u16

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

**ZERO WIDTH NO-BREAK SPACE**

utf-16 인코딩은 빅엔디언과 리틀엔디언 cpu 간의 혼란을 방지하기 위해 이 문자를 인코딩된 텍스트 앞에 붙임.  
리틀엔디언 또는 빅엔디언을 명시하는 UTF-16LE, UTF-16BE 변형을 사용할 경우 BOM 생성 X


In [47]:
# little endian
u16le = 'El Niño'.encode('utf_16le')
u16le

b'E\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

In [48]:
# big endian
u16be = 'El Niño'.encode('utf_16be')
u16be

b'\x00E\x00l\x00 \x00N\x00i\x00\xf1\x00o'

In [46]:
u16le

b'E\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

## Handling Text Files

**유니코드 샌드위치**:  텍스트를 처리할 때 최고의 순서

1. bytes를 str로 변환
2. str 객체로만 작업
3. str 객체를 bytes로 변환

#### Example 4-9. A platform encoding issue

In [53]:
open('cafe.txt', 'w', encoding='utf_8').write('café')
open('cafe.txt', encoding='cp1252').read()

'cafÃ©'

#### Example 4-10. Closer inspection of Example 4-9 running on Windows reveals the bug and how to fix it

In [64]:
fp = open('cafe.txt', 'w', encoding='utf-8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf-8'>

In [65]:
fp.write('café')
fp.close()

In [66]:
import os
os.stat('cafe.txt').st_size

5

In [67]:
fp2 = open('cafe.txt', encoding='cp1252')
fp2

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp1252'>

In [68]:
fp2.read()

'cafÃ©'

In [69]:
fp3 = open('cafe.txt', encoding='utf_8')
fp3

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf_8'>

In [70]:
fp3.read()

'café'

In [71]:
fp4 = open('cafe.txt', 'rb')
fp4.read()

b'caf\xc3\xa9'

### Encoding Defaults: Madhouse

#### Example 4-11. Exploring encoding defaults

In [77]:
import sys, locale

expressions = """
  locale.getpreferredencoding()
  type(my_file)
  my_file.encoding
  sys.stdout.isatty()
  sys.stdout.encoding
  sys.stdin.isatty()
  sys.stdin.encoding
  sys.stderr.isatty()
  sys.stderr.encoding
  sys.getdefaultencoding()
  sys.getfilesystemencoding()
"""

my_file = open('dummy', 'w')

for expression in expressions.split():
  value = eval(expression)
  print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF-8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


In [76]:
eval("locale.getpreferredencoding()")

'UTF-8'

## Normalizing Unicode for Saner Comparisons

유니코드는 결합 문자가 있어 문자열 비교가 간단하지 않음. (출력은 같으나 코드 포인트가 다름)  
unicodedata.normalize()를 이용해 유니코드 정규화가 필요.  

정규화 방법  
* NFC: 코드 포인트를 조합해서 가장 짧은 동일 문자열 생성.  
* NFD: 조합된 문자를 기본 문자와 별도의 결합 문자로 분리.
* NFKC, NFKD: 호환성 문자에 영향을 미침 -> 다른 표준(e.g. µ, ℀)의 상호변환을 지원하기 위한 문자

In [78]:
s1 = 'café'
s2 = 'cafe\u0301'
print(s1, s2)
print(len(s1), len(s2))
print(s1==s2)

café café
4 5
False


In [85]:
# 결합문자
from unicodedata import normalize
s1 = 'café'
s2 = 'cafe\u0301'
print(len(normalize('NFC', s1)), len(normalize('NFC', s2)))
print(len(normalize('NFD', s1)), len(normalize('NFD', s2)))
print(normalize('NFC', s1) == normalize('NFC', s2))
print(normalize('NFD', s1) == normalize('NFD', s2))

4 4
5 5
True
True


In [86]:
# 단일문자
from unicodedata import normalize, name
ohm = '\u2126'
print(name(ohm))

ohm_c = normalize('NFC', ohm)
print(name(ohm_c))

print(ohm == ohm_c)
print(normalize('NFC', ohm) == normalize('NFC', ohm))

OHM SIGN
GREEK CAPITAL LETTER OMEGA
False
True


In [87]:
# NFKC
from unicodedata import normalize, name

half = '½'
print(normalize('NFKC', half))
four_squared = '4²'
print(normalize('NFKC', four_squared))

micro = 'µ'
micro_kc = normalize('NFKC', micro)
print(micro, micro_kc)
print(ord(micro), ord(micro_kc))
print(name(micro), name(micro_kc))

1⁄2
42
µ μ
181 956
MICRO SIGN GREEK SMALL LETTER MU
